In [2]:
import keras
import keras.layers as layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import cv2
from keras.models import Model
from keras.utils import plot_model
from google.colab.patches import cv2_imshow

In [7]:
count = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
train_images_MVS = []
train_images_DSV = []
Depth_gt = []
for i in range(1, 13):
  train_images_MVS.append(np.concatenate((mpimg.imread("Files/cam ({}).jpg".format(i)),
                                          cv2.resize(mpimg.imread("Files/mvs{}.png".format(i)), (960, 540))),
                                          axis = 2))
  train_images_DSV.append(np.concatenate((mpimg.imread("Files/cam ({}).jpg".format(i)),
                                          np.stack((mpimg.imread("Files/DSV ({}).png".format(i)),)*3, axis=-1)*255),
                                          axis = 2))
  Depth_gt.append(np.load("Files/cam ({}).npy".format(i)))

train_images_MVS = np.array(train_images_MVS)
train_images_DSV = np.array(train_images_DSV)
Depth_gt = np.array(Depth_gt).reshape(12, 540, 960, 1)
print(Depth_gt.shape)
print(train_images_MVS.shape)
print(train_images_DSV.shape)

(12, 540, 960, 1)
(12, 540, 960, 6)
(12, 540, 960, 6)


In [21]:
keras.backend.clear_session()

DSV = keras.Input(shape=(540, 960, 6))
DMV = keras.Input(shape=(540, 960, 6))

l1 = layers.Conv2D(64, kernel_size = (3, 3), padding = 'same')(DSV)
l1 = layers.LeakyReLU(alpha = 0.2)(l1)
l1 = layers.Conv2D(64, kernel_size = (3, 3), padding = 'same')(l1)
l1 = layers.LeakyReLU(alpha = 0.2)(l1)

l1m = layers.MaxPool2D()(l1)

l2 = layers.Conv2D(128, kernel_size = (3, 3), padding = 'same')(l1m)
l2 = layers.LeakyReLU(alpha = 0.2)(l2)
l2 = layers.Conv2D(128, kernel_size = (3, 3), padding = 'same')(l2)
l2 = layers.LeakyReLU(alpha = 0.2)(l2)

l2m = layers.MaxPool2D()(l2)

l3 = layers.Conv2D(256, kernel_size = (3, 3), padding = 'same')(l2m)
l3 = layers.LeakyReLU(alpha = 0.2)(l3)
l3 = keras.layers.ZeroPadding2D(padding=((1, 0), (0, 0)))(l3)
l3 = layers.Conv2D(256, kernel_size = (3, 3), padding = 'same')(l3)
l3 = layers.LeakyReLU(alpha = 0.2)(l3)

l3m = layers.MaxPool2D()(l3)

l4 = layers.Conv2D(512, kernel_size = (3, 3), padding = 'same')(l3m)
l4 = layers.LeakyReLU(alpha = 0.2)(l4)
l4 = layers.Conv2D(512, kernel_size = (3, 3), padding = 'same')(l4)
l4 = layers.LeakyReLU(alpha = 0.2)(l4)

l4m = layers.MaxPool2D()(l4)

l5 = layers.Conv2D(512, kernel_size = (3, 3), padding = 'same')(l4m)
l5 = layers.LeakyReLU(alpha = 0.2)(l5)
l5 = layers.Conv2D(512, kernel_size = (3, 3), padding = 'same')(l5)
l5 = layers.LeakyReLU(alpha = 0.2)(l5)

l5m = layers.MaxPool2D()(l5)

l6 = layers.Conv2D(64, kernel_size = (3, 3), padding = 'same')(DMV)
l6 = layers.LeakyReLU(alpha = 0.2)(l6)
l6 = layers.Conv2D(64, kernel_size = (3, 3), padding = 'same')(l6)
l6 = layers.LeakyReLU(alpha = 0.2)(l6)

l6m = layers.MaxPool2D()(l6)

l7 = layers.Conv2D(128, kernel_size = (3, 3), padding = 'same')(l6m)
l7 = layers.LeakyReLU(alpha = 0.2)(l7)
l7 = layers.Conv2D(128, kernel_size = (3, 3), padding = 'same')(l7)
l7 = layers.LeakyReLU(alpha = 0.2)(l7)

l7m = layers.MaxPool2D()(l7)

l8 = layers.Conv2D(256, kernel_size = (3, 3), padding = 'same')(l7m)
l8 = layers.LeakyReLU(alpha = 0.2)(l8)
l8 = keras.layers.ZeroPadding2D(padding=((1, 0), (0, 0)))(l8)
l8 = layers.Conv2D(256, kernel_size = (3, 3), padding = 'same')(l8)
l8 = layers.LeakyReLU(alpha = 0.2)(l8)

l8m = layers.MaxPool2D()(l8)

l9 = layers.Conv2D(512, kernel_size = (3, 3), padding = 'same')(l8m)
l9 = layers.LeakyReLU(alpha = 0.2)(l9)
l9 = layers.Conv2D(512, kernel_size = (3, 3), padding = 'same')(l9)
l9 = layers.LeakyReLU(alpha = 0.2)(l9)

l9m = layers.MaxPool2D()(l9)

l10 = layers.Conv2D(512, kernel_size = (3, 3), padding = 'same')(l9m)
l10 = layers.LeakyReLU(alpha = 0.2)(l10)
l10 = layers.Conv2D(512, kernel_size = (3, 3), padding = 'same')(l10)
l10 = layers.LeakyReLU(alpha = 0.2)(l10)

l10m = layers.MaxPool2D()(l10)

l11 = layers.merge.concatenate([l5m, l10m])

l12 = layers.Conv2D(512, kernel_size = (3, 3), padding = 'same')(l11)
l12 = layers.LeakyReLU(alpha = 0.2)(l12)
l12 = layers.Conv2DTranspose(256, kernel_size = (3, 3), strides = (2, 2), padding = 'same')(l12)
l12 = layers.LeakyReLU(alpha = 0.2)(l12)

l12m = layers.merge.concatenate([l10, l5, l12])

l13 = layers.Conv2D(512, kernel_size = (3, 3), padding = 'same')(l12m)
l13 = layers.LeakyReLU(alpha = 0.2)(l13)
l13 = layers.Conv2DTranspose(256, kernel_size = (3, 3), strides = (2, 2), padding = 'same')(l13)
l13 = layers.LeakyReLU(alpha = 0.2)(l13)

l13m = layers.merge.concatenate([l9, l4, l13])

l14 = layers.Conv2D(512, kernel_size = (3, 3), padding = 'same')(l13m)
l14 = layers.LeakyReLU(alpha = 0.2)(l14)
l14 = layers.Conv2DTranspose(256, kernel_size = (3, 3), strides = (2, 2), padding = 'same')(l14)
l14 = layers.LeakyReLU(alpha = 0.2)(l14)

l14m = layers.merge.concatenate([l8, l3, l14])

l15 = layers.Conv2D(256, kernel_size = (3, 3), padding = 'same')(l14m)
l15 = layers.LeakyReLU(alpha = 0.2)(l15)
l15 = layers.Conv2DTranspose(64, kernel_size = (3, 3), strides = (2, 2), padding = 'same')(l15[:,1:,:,:])
l15 = layers.LeakyReLU(alpha = 0.2)(l15)

l15m = layers.merge.concatenate([l7, l2, l15])

l16 = layers.Conv2D(128, kernel_size = (3, 3), padding = 'same')(l15m)
l16 = layers.LeakyReLU(alpha = 0.2)(l16)
l16 = layers.convolutional.Deconv2D(32, kernel_size = (3, 3), strides = (2, 2), padding = 'same')(l16)
l16 = layers.LeakyReLU(alpha = 0.2)(l16)

l16m = layers.merge.concatenate([l6, l1, l16])

l17 = layers.Conv2D(32, kernel_size = (3, 3), padding = 'same')(l16m)
l17 = layers.LeakyReLU(alpha = 0.2)(l17)

l18 = layers.Conv2D(1, kernel_size = (3, 3), padding = 'same')(l17)

model = Model(inputs = [DSV, DMV], outputs = l18)
model.summary()

plot_model(model, to_file = 'Depth_Fusion.png', show_layer_names=False)

model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 540, 960, 6) 0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 540, 960, 6) 0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 540, 960, 64) 3520        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 540, 960, 64) 3520        input_1[0][0]                    
_______________________________________________________________________________________

In [9]:
history = model.fit([train_images_DSV, train_images_MVS],
                    Depth_gt, epochs = 10, verbose = 1)

Epoch 1/10
1/1 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 8/10
1/1 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 9/10
1/1 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 10/10
1/1 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.0000e+00
